In [1]:
# Cell 1: Setup imports
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Thêm đường dẫn gốc của dự án vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import TradingEngine và components
from src.core.trading_engine import TradingEngine
from src.strategies.sma_crossover import SMACrossoverStrategy
from src.strategies.rsi_strategy import RSIStrategy
from src.strategies.macd_strategy import MACDStrategy
from src.utils.config_manager import ConfigManager
from src.data.data_manager import DataManager

# Import visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

print("Import thành công!")

C:\Users\huynh.th\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\vnai\scope\profile.py:562: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Import thành công!


In [2]:
# Cell 2: Khởi tạo TradingEngine
config_path = "../../config/config.yaml"
engine = TradingEngine(config_path)

print(f"TradingEngine đã được khởi tạo với vốn ban đầu: ${engine.cash:,.2f}")
print(f"Config loaded: {engine.config.get('trading.symbols')}")
print(f"Config loaded: {engine.config.get('data.source')}")

2025-07-31 17:30:13.811 | INFO     | src.utils.config_manager:__init__:31 - Configuration loaded from ../../config/config.yaml
2025-07-31 17:30:13.812 | INFO     | src.risk.risk_manager:__init__:49 - Risk manager initialized
2025-07-31 17:30:13.812 | INFO     | src.core.trading_engine:__init__:76 - Trading engine initialized with $100,000.00 initial capital


TradingEngine đã được khởi tạo với vốn ban đầu: $100,000.00
Config loaded: ['Bitstamp:BTCUSD']
Config loaded: tradingview


In [4]:
# Cell 3: Chuẩn bị data
# Sử dụng data có sẵn hoặc lấy từ DataManager
symbols = engine.config.get("trading.symbols", ["AAPL"])
start_date = engine.config.get("data.start_date", "2023-01-01")
end_date = engine.config.get("data.end_date", "2023-12-31")

print(f"Lấy data cho symbols: {symbols}")
print(f"Period: {start_date} đến {end_date}")

# Lấy data từ DataManager
data_manager = DataManager(engine.config)
historical_data = data_manager.get_historical_data(
    symbols=symbols,
    start_date=start_date,
    end_date=end_date,
    interval=engine.config.get("data.interval", "1d")
)

print(f"Data shape: {historical_data.shape}")
print(f"Data columns: {historical_data.columns.tolist()}")
print(f"Date range: {historical_data.index.min()} đến {historical_data.index.max()}")

historical_data

2025-07-31 17:30:19.781 | INFO     | src.data.data_manager:get_historical_data:60 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31


Lấy data cho symbols: ['Bitstamp:BTCUSD']
Period: 2023-01-01 đến 2025-07-31


2025-07-31 17:30:20.346 | INFO     | src.data.data_manager:get_historical_data:93 - OHLCV data cached successfully


Data shape: (1000, 6)
Data columns: ['time', 'open', 'high', 'low', 'close', 'volume']
Date range: 0 đến 999


,time,open,high,low,close,volume
0,2022-11-05 07:00:00,21152.0,21473.0,21089.0,21301.0,774.125220
1,2022-11-06 07:00:00,21299.0,21364.0,20899.0,20907.0,551.958490
2,2022-11-07 07:00:00,20899.0,21070.0,20400.0,20590.0,1417.995635
3,2022-11-08 07:00:00,20591.0,20669.0,17114.0,18550.0,12038.833212
4,2022-11-09 07:00:00,18546.0,18588.0,15632.0,15877.0,8350.789461
...,...,...,...,...,...,...
995,2025-07-27 07:00:00,117983.0,119798.0,117888.0,119469.0,579.574775
996,2025-07-28 07:00:00,119514.0,119826.0,117400.0,118051.0,1062.351735
997,2025-07-29 07:00:00,118068.0,119267.0,116928.0,117926.0,1156.649402
998,2025-07-30 07:00:00,117926.0,118775.0,115784.0,117834.0,1043.517072


In [5]:
# Cell 4: Thêm strategies
# SMA Crossover Strategy
sma_strategy = SMACrossoverStrategy({
    "short_window": 10,
    "long_window": 30,
    "name": "SMA_Crossover"
})
engine.add_strategy(sma_strategy)

# RSI Strategy
rsi_strategy = RSIStrategy({
    "period": 14,
    "oversold": 30,
    "overbought": 70,
    "name": "RSI_Strategy"
})
engine.add_strategy(rsi_strategy)

# MACD Strategy
macd_strategy = MACDStrategy({
    "fast_period": 12,
    "slow_period": 26,
    "signal_period": 9,
    "name": "MACD_Strategy"
})
engine.add_strategy(macd_strategy)

print(f"Đã thêm {len(engine.strategies)} strategies:")
for name, strategy in engine.strategies.items():
    print(f"  - {name}: {strategy.__class__.__name__}")

2025-07-31 17:30:22.837 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-07-31 17:30:22.838 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 10/30
2025-07-31 17:30:22.838 | INFO     | src.core.trading_engine:add_strategy:81 - Added strategy: sma_crossover
2025-07-31 17:30:22.839 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-07-31 17:30:22.839 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=14, overbought=70, oversold=30
2025-07-31 17:30:22.840 | INFO     | src.core.trading_engine:add_strategy:81 - Added strategy: rsi
2025-07-31 17:30:22.840 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-07-31 17:30:22.841 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=12, slow=26, signal=9
2025-07-31 17:30:22.841 | INFO     | src.core.trading_engine:add

Đã thêm 3 strategies:
  - sma_crossover: SMACrossoverStrategy
  - rsi: RSIStrategy
  - macd: MACDStrategy


In [6]:
# Cell 5: Chạy backtest
print("Bắt đầu chạy backtest...")
print(f"Period: {start_date} đến {end_date}")

# Chạy backtest
engine.run_backtest(start_date, end_date)

2025-07-31 17:30:24.540 | INFO     | src.core.trading_engine:run_backtest:85 - Starting backtest from 2023-01-01 to 2025-07-31
2025-07-31 17:30:24.540 | INFO     | src.data.data_manager:get_historical_data:60 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31


Bắt đầu chạy backtest...
Period: 2023-01-01 đến 2025-07-31


2025-07-31 17:30:25.034 | INFO     | src.data.data_manager:get_historical_data:93 - OHLCV data cached successfully
2025-07-31 17:30:25.041 | INFO     | src.core.trading_engine:run_backtest:106 - Loaded 1000 data points for ['time', 'open', 'high', 'low', 'close', 'volume']
2025-07-31 17:30:25.061 | INFO     | src.strategies.rsi_strategy:_generate_signal_for_symbol:140 - RSI sustained overbought for time: 100.00
2025-07-31 17:30:25.064 | INFO     | src.strategies.rsi_strategy:_generate_signal_for_symbol:135 - RSI sustained oversold for open: 24.74
2025-07-31 17:30:25.066 | INFO     | src.strategies.rsi_strategy:_generate_signal_for_symbol:135 - RSI sustained oversold for high: 7.87
2025-07-31 17:30:25.067 | INFO     | src.strategies.rsi_strategy:_generate_signal_for_symbol:135 - RSI sustained oversold for low: 22.16
2025-07-31 17:30:25.068 | INFO     | src.strategies.rsi_strategy:_generate_signal_for_symbol:135 - RSI sustained oversold for close: 24.74
2025-07-31 17:30:25.069 | INFO    

In [7]:
# Cell 6: Lấy kết quả backtest
portfolio_summary = engine.get_portfolio_summary()

print("=== KẾT QUẢ BACKTEST ===")
print(f"Initial Capital: ${portfolio_summary['initial_capital']:,.2f}")
print(f"Final Portfolio Value: ${portfolio_summary['total_value']:,.2f}")
print(f"Total Return: {portfolio_summary['total_return']:.2%}")
print(f"Annualized Return: {portfolio_summary['annualized_return']:.2%}")
print(f"Sharpe Ratio: {portfolio_summary['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {portfolio_summary['max_drawdown']:.2%}")
print(f"Win Rate: {portfolio_summary['win_rate']:.2%}")
print(f"Total Trades: {portfolio_summary['total_trades']}")

=== KẾT QUẢ BACKTEST ===
Initial Capital: $100,000.00
Final Portfolio Value: $195.96
Total Return: -99.80%
Annualized Return: -79.25%
Sharpe Ratio: -1.17
Max Drawdown: -99.82%
Win Rate: 53.40%
Total Trades: 286


In [8]:
# Cell 7: Visualize portfolio performance
def plot_portfolio_performance(engine):
    """Plot portfolio performance"""
    portfolio_history = engine.portfolio_history
    
    if not portfolio_history:
        print("Không có dữ liệu portfolio history")
        return
    
    df = pd.DataFrame(portfolio_history)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('Portfolio Value', 'Daily Returns', 'Cumulative Returns', 
                       'Drawdown', 'Cash vs Positions', 'Trade Distribution'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Portfolio Value
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'], 
                  mode='lines', name='Portfolio Value'),
        row=1, col=1
    )
    
    # Daily Returns
    daily_returns = df['total_value'].pct_change()
    fig.add_trace(
        go.Scatter(x=df.index, y=daily_returns, 
                  mode='lines', name='Daily Returns'),
        row=1, col=2
    )
    
    # Cumulative Returns
    cumulative_returns = (1 + daily_returns).cumprod()
    fig.add_trace(
        go.Scatter(x=df.index, y=cumulative_returns, 
                  mode='lines', name='Cumulative Returns'),
        row=2, col=1
    )
    
    # Drawdown
    running_max = df['total_value'].expanding().max()
    drawdown = (df['total_value'] - running_max) / running_max
    fig.add_trace(
        go.Scatter(x=df.index, y=drawdown, 
                  mode='lines', name='Drawdown', fill='tonexty'),
        row=2, col=2
    )
    
    # Cash vs Positions
    fig.add_trace(
        go.Scatter(x=df.index, y=df['cash'], 
                  mode='lines', name='Cash'),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'] - df['cash'], 
                  mode='lines', name='Positions'),
        row=3, col=1
    )
    
    # Trade Distribution (if available)
    if hasattr(engine, 'trades') and engine.trades:
        trade_returns = [trade.pnl for trade in engine.trades]
        fig.add_trace(
            go.Histogram(x=trade_returns, name='Trade Returns'),
            row=3, col=2
        )
    
    fig.update_layout(height=900, title_text="Portfolio Performance Analysis")
    fig.show()

# Chạy visualization
plot_portfolio_performance(engine)

In [9]:
# Cell 8: Hiển thị chi tiết trades
if hasattr(engine, 'trades') and engine.trades:
    print(f"\n=== CHI TIẾT TRADES ({len(engine.trades)} trades) ===")
    
    trades_df = pd.DataFrame([
        {
            'Symbol': trade.symbol,
            'Side': trade.side,
            'Quantity': trade.quantity,
            'Price': trade.price,
            'Timestamp': trade.timestamp,
            'Commission': trade.commission,
            'Strategy': trade.strategy
        }
        for trade in engine.trades
    ])
    
    print(trades_df.head(10))
    
    # Trade statistics
    print(f"\n=== TRADE STATISTICS ===")
    print(f"Total Trades: {len(engine.trades)}")
    print(f"Buy Trades: {len(trades_df[trades_df['Side'] == 'buy'])}")
    print(f"Sell Trades: {len(trades_df[trades_df['Side'] == 'sell'])}")
    print(f"Average Trade Size: {trades_df['Quantity'].mean():.2f}")
    print(f"Total Commission: ${trades_df['Commission'].sum():.2f}")
else:
    print("Không có trades nào được thực hiện")


=== CHI TIẾT TRADES (286 trades) ===
  Symbol Side  Quantity    Price  Timestamp  Commission  Strategy
0   open  buy  1.198538  16687.0         15   20.000000  strategy
1   high  buy  0.956127  16730.0         15   15.996000  strategy
2    low  buy  0.791243  16169.0         15   12.793601  strategy
3  close  buy  0.629449  16256.0         15   10.232322  strategy
4   open  buy  0.503619  16250.0         16    8.183811  strategy
5   high  buy  0.402324  16269.0         16    6.545412  strategy
6    low  buy  0.338201  15479.0         16    5.235021  strategy
7  close  buy  0.265570  15766.0         16    4.186969  strategy
8   open  buy  0.212524  15757.0         17    3.348738  strategy
9   high  buy  0.164506  16281.0         17    2.678321  strategy

=== TRADE STATISTICS ===
Total Trades: 286
Buy Trades: 183
Sell Trades: 103
Average Trade Size: 0.93
Total Commission: $1625.51


In [10]:
# Cell 9: So sánh performance của các strategies
def compare_strategies(engine):
    """Compare performance of different strategies"""
    if not hasattr(engine, 'strategies') or not engine.strategies:
        print("Không có strategies để so sánh")
        return
    
    strategy_results = {}
    
    for name, strategy in engine.strategies.items():
        # Calculate basic metrics for each strategy
        # This is a simplified version - you might want to run separate backtests
        strategy_results[name] = {
            'name': name,
            'type': strategy.__class__.__name__,
            'parameters': strategy.get_summary()
        }
    
    # Display strategy comparison
    print("=== STRATEGY COMPARISON ===")
    for name, result in strategy_results.items():
        print(f"\nStrategy: {name}")
        print(f"Type: {result['type']}")
        print(f"Parameters: {result['parameters']}")

compare_strategies(engine)

=== STRATEGY COMPARISON ===

Strategy: sma_crossover
Type: SMACrossoverStrategy
Parameters: {'name': 'sma_crossover', 'strategy_type': 'SMA Crossover', 'short_window': 10, 'long_window': 30, 'description': 'SMA Crossover with 10/30 periods'}

Strategy: rsi
Type: RSIStrategy
Parameters: {'name': 'rsi', 'strategy_type': 'RSI Strategy', 'period': 14, 'overbought_threshold': 70, 'oversold_threshold': 30, 'confirmation_period': 2, 'description': 'RSI Strategy with 14 period, 30/70 levels'}

Strategy: macd
Type: MACDStrategy
Parameters: {'name': 'macd', 'strategy_type': 'MACD Strategy', 'fast_period': 12, 'slow_period': 26, 'signal_period': 9, 'histogram_threshold': 0.0, 'confirmation_period': 1, 'description': 'MACD Strategy with 12/26/9 periods'}
